<a href="https://colab.research.google.com/github/ll1195831146/Infor7374-AI/blob/master/Assignment2/E5%266_Transfer_Learning_and_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import keras
from keras_applications.imagenet_utils import _obtain_input_shape
from keras import backend as K
from keras.layers import Input, Convolution2D, \
    GlobalAveragePooling2D, Dense, BatchNormalization, Activation
from keras.models import Model
from keras.metrics import top_k_categorical_accuracy, categorical_accuracy
from keras.engine.topology import get_source_inputs
from keras.layers import DepthwiseConv2D
from keras.applications.mobilenet import MobileNet
from keras.preprocessing.image import ImageDataGenerator
from keras.datasets import cifar10 
import numpy as np
from skimage.transform import resize
import os

In [0]:
#base setup
batch_size = 32
num_classes = 10
epochs = 5
data_augmentation = True
subtract_pixel_mean = True
num_predictions = 20
size = 224
alpha = 1.0

In [0]:
def resize_image_arr(img_arr):
    x_resized_list = []
    for i in range(img_arr.shape[0]):
        img = img_arr[0]
        resized_img = resize(img, (size, size))
        x_resized_list.append(resized_img)
    return np.stack(x_resized_list)

In [0]:
# Load the CIFAR10 data.
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train[0:2000]
y_train = y_train[0:2000]
x_test = x_test[0:500]
y_test = y_test[0:500]

In [0]:
# Resize image arrays
x_train = resize_image_arr(x_train)
x_test = resize_image_arr(x_test)

/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [0]:
# Input image dimensions.
input_shape = x_train.shape[1:]

# Normalize data.
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

In [0]:
# If subtract pixel mean is enabled
if subtract_pixel_mean:
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('y_train shape:', y_train.shape)

x_train shape: (2000, 224, 224, 3)
2000 train samples
500 test samples
y_train shape: (2000, 1)


In [0]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
model = MobileNet(input_shape=input_shape, weights='imagenet', include_top=False, alpha=alpha)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
__________

In [0]:
# evaluate_on_imagenet

# eval_data = model.evaluate_generator(datagen.flow_from_directory(
#            '/root/imagenet/validation',
#            target_size=(edge_size, edge_size),
#            batch_size=BATCH_SIZE,
#            class_mode='categorical'),steps=100)
#     for i, metric_name in enumerate(base_model.metrics_names):
#        print "   - {0}: {1}".format(metric_name, eval_data[i])

In [0]:
# add a global spatial average pooling layer
x = model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(512, activation='relu')(x)
# and a logistic layer -- 10 classes for CIFAR10
predictions = Dense(num_classes, activation='softmax')(x)

In [0]:
# this is the model we will train
model = Model(inputs=model.input, outputs=predictions)

In [0]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

In [0]:
# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=[categorical_accuracy, top_k_categorical_accuracy])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [0]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional layers
for layer in model.layers:
    layer.trainable = False

In [0]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,steps_per_epoch=x_train.shape[0] // batch_size,
                        validation_data=(x_test, y_test),
                        workers=4)

Using real-time data augmentation.


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Epoch 1/5
62/62 [==============================] - 32s 515ms/step - loss: 2.4317 - categorical_accuracy: 0.0998 - top_k_categorical_accuracy: 0.4980 - val_loss: 2.4439 - val_categorical_accuracy: 0.1140 - val_top_k_categorical_accuracy: 0.5020
Epoch 2/5
62/62 [==============================] - 23s 367ms/step - loss: 2.4198 - categorical_accuracy: 0.0932 - top_k_categorical_accuracy: 0.4929 - val_loss: 2.3373 - val_categorical_accuracy: 0.1140 - val_top_k_categorical_accuracy: 0.5160
Epoch 3/5
62/62 [==============================] - 23s 364ms/step - loss: 2.4004 - categorical_accuracy: 0.0948 - top_k_categorical_accuracy: 0.5116 - val_loss: 2.3914 - val_categorical_accuracy: 0.0800 - val_top_k_categorical_accuracy: 0.5120
Epoch 4/5
62/62 [==============================] - 23s 365ms/step - loss: 2.3859 - categorical_accuracy: 0.1048 - top_k_categorical_accuracy: 0.5156 - val_loss: 2.4244 - val_categorical_accuracy: 0.1140 - val_top_k_categorical_accuracy: 0.4820
Epoch 5/5
62/62 [=======

In [0]:
# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

500/500 [==============================] - 2s 4ms/step
Test loss: 2.496279285430908
Test accuracy: 0.098


**Fine-tuning**
*   New dataset is small and similar to original dataset. Since the data is small, it is not a good idea to fine-tune the ConvNet due to overfitting concerns. Since the data is similar to the original data, we expect higher-level features in the ConvNet to be relevant to this dataset as well. Hence, the best idea might be to train a linear classifier on the CNN codes.
*   New dataset is large and similar to the original dataset. Since we have more data, we can have more confidence that we won’t overfit if we were to try to fine-tune through the full network.
*   New dataset is small but very different from the original dataset. Since the data is small, it is likely best to only train a linear classifier. Since the dataset is very different, it might not be best to train the classifier form the top of the network, which contains more dataset-specific features. Instead, it might work better to train the SVM classifier from activations somewhere earlier in the network.
*   New dataset is large and very different from the original dataset. Since the dataset is very large, we may expect that we can afford to train a ConvNet from scratch. However, in practice it is very often still beneficial to initialize with weights from a pretrained model. In this case, we would have enough data and confidence to fine-tune through the entire network.


In [0]:
# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(model.layers):
    print(i, layer.name)

0 input_4
1 conv1_pad
2 conv1
3 conv1_bn
4 conv1_relu
5 conv_dw_1
6 conv_dw_1_bn
7 conv_dw_1_relu
8 conv_pw_1
9 conv_pw_1_bn
10 conv_pw_1_relu
11 conv_pad_2
12 conv_dw_2
13 conv_dw_2_bn
14 conv_dw_2_relu
15 conv_pw_2
16 conv_pw_2_bn
17 conv_pw_2_relu
18 conv_dw_3
19 conv_dw_3_bn
20 conv_dw_3_relu
21 conv_pw_3
22 conv_pw_3_bn
23 conv_pw_3_relu
24 conv_pad_4
25 conv_dw_4
26 conv_dw_4_bn
27 conv_dw_4_relu
28 conv_pw_4
29 conv_pw_4_bn
30 conv_pw_4_relu
31 conv_dw_5
32 conv_dw_5_bn
33 conv_dw_5_relu
34 conv_pw_5
35 conv_pw_5_bn
36 conv_pw_5_relu
37 conv_pad_6
38 conv_dw_6
39 conv_dw_6_bn
40 conv_dw_6_relu
41 conv_pw_6
42 conv_pw_6_bn
43 conv_pw_6_relu
44 conv_dw_7
45 conv_dw_7_bn
46 conv_dw_7_relu
47 conv_pw_7
48 conv_pw_7_bn
49 conv_pw_7_relu
50 conv_dw_8
51 conv_dw_8_bn
52 conv_dw_8_relu
53 conv_pw_8
54 conv_pw_8_bn
55 conv_pw_8_relu
56 conv_dw_9
57 conv_dw_9_bn
58 conv_dw_9_relu
59 conv_pw_9
60 conv_pw_9_bn
61 conv_pw_9_relu
62 conv_dw_10
63 conv_dw_10_bn
64 conv_dw_10_relu
65 conv_pw_10

In [0]:
# We will finetune from add_9
for layer in model.layers[:99]:
    layer.trainable = False
for layer in model.layers[99:]:
    layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), 
              loss='categorical_crossentropy',
              metrics=[categorical_accuracy, top_k_categorical_accuracy])

# Train!
train_model = model.fit_generator(
    datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,steps_per_epoch=x_train.shape[0] // batch_size,
                        validation_data=(x_test, y_test),
                        workers=4)


Epoch 1/5
62/62 [==============================] - 26s 413ms/step - loss: 2.3591 - categorical_accuracy: 0.1013 - top_k_categorical_accuracy: 0.5086 - val_loss: 2.4963 - val_categorical_accuracy: 0.0980 - val_top_k_categorical_accuracy: 0.5120
Epoch 2/5
62/62 [==============================] - 18s 287ms/step - loss: 2.3619 - categorical_accuracy: 0.1018 - top_k_categorical_accuracy: 0.5070 - val_loss: 2.4963 - val_categorical_accuracy: 0.0980 - val_top_k_categorical_accuracy: 0.5120
Epoch 3/5
62/62 [==============================] - 19s 310ms/step - loss: 2.3580 - categorical_accuracy: 0.1028 - top_k_categorical_accuracy: 0.5131 - val_loss: 2.4963 - val_categorical_accuracy: 0.0980 - val_top_k_categorical_accuracy: 0.5120
Epoch 4/5
62/62 [==============================] - 19s 310ms/step - loss: 2.3627 - categorical_accuracy: 0.0988 - top_k_categorical_accuracy: 0.5051 - val_loss: 2.4963 - val_categorical_accuracy: 0.0980 - val_top_k_categorical_accuracy: 0.5120
Epoch 5/5
62/62 [=======

In [0]:
# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[2])

500/500 [==============================] - 2s 3ms/step
Test loss: 2.496279285430908
Test accuracy: 0.5120000002384186
